<a href="https://colab.research.google.com/github/pipeberrio/Analitica/blob/main/Tarea4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TAREA NUMERO 4**


FELIPE BERRIO GOMEZ CC1035416370

<p><img alt="banner" height="252px" width="1080px" src="https://docs.google.com/uc?export=download&id=1SqbMIjMfsMlSukiTyfMJ0VMuDlP2hGhx"  align="center" hspace="10px" vspace="0px" ></p>

# <font color='056938'> **Descripción del problema** </font>

La Autoridad Nacional de Licencias Ambientales ha consolidado y preprocesado la siguiente base de datos respecto a los trabsformadores de material reciclado en Colombia.










In [ ]:
import pandas as pd


!gdown 1oP0ygviQAsthXGlzTJnknd72F6mHPhUV
df = pd.read_csv('datos_transformadores.csv')
df.head()


Con el finde construir los insumos necesarios para generar un tablero de datos, se requiere que usted diseñe algunas funciones para crear los gráficos que iran en el

## <font color='8EC044'> **Ejercicio 1** </font>

Diseñe una función que dados el argumento `material` retorne un mapa de colombia visualizando todos los transformadores que realizan procesamiento de dicho material. La función deberá tener un valor por defecto todos, que muestre todos los transformadores sin impoertar le material que procesan. Los posible valores para el argunmento material son `['plástico', 'cartón', 'vidrio', 'metales', 'multimaterial', 'varios', 'papel']`


In [ ]:
!gdown 1yyOPsDXgSMJcISDH-_7HqNvqoynnOcTQ # Descargar el archivo co_2018_MGN_MPIO_POLITICO.geojson

import pandas as pd
import geopandas as gpd
import plotly.express as px

# Cargar el archivo de transformadores
df = pd.read_csv('datos_transformadores.csv')

# Cargar el archivo GeoJSON de Colombia
gdf_colombia = gpd.read_file('co_2018_MGN_MPIO_POLITICO.geojson')

def graficar_transformadores_mapa(material="todos"):
    """
    Función para visualizar los transformadores de material reciclado en un mapa de Colombia.

    Parámetros:
    - material (str): Tipo de material a mostrar. Puede ser 'plástico', 'cartón', 'vidrio', 'metales',
                      'multimaterial', 'varios', 'papel' o 'todos' (por defecto).
    """
    # Filtrar los datos según el tipo de material
    if material != "todos":
        df_filtrado = df[df['material'] == material]
    else:
        df_filtrado = df

    # Convertir el DataFrame en un GeoDataFrame para asegurar coordenadas geográficas
    gdf_transformadores = gpd.GeoDataFrame(
        df_filtrado,
        geometry=gpd.points_from_xy(df_filtrado['longitud'], df_filtrado['latitud']),
        crs="EPSG:4326"
    )

    # Convertir el GeoDataFrame de Colombia a formato GeoJSON
    geojson_colombia = gdf_colombia.__geo_interface__

    # Crear el mapa de Colombia
    fig = px.scatter_mapbox(
        gdf_transformadores,
        lat=gdf_transformadores.geometry.y,
        lon=gdf_transformadores.geometry.x,
        hover_name="razon social",
        hover_data={
            "departamento": True,
            "capacidad": True,
            "material": True
        },
        color="material",
        color_discrete_sequence=px.colors.qualitative.Set3,
        zoom=4.5,
        center={"lat": 4.5709, "lon": -74.2973},
        opacity=0.9,
        size_max=10,  # Aumentar el tamaño máximo de los puntos
        size=[7] * len(gdf_transformadores)  # Tamaño consistente de los puntos para mayor visibilidad
    )

    # Añadir el estilo de mapa y configuraciones
    fig.update_layout(
        mapbox_style="carto-positron",
        mapbox_layers=[
            {
                "source": geojson_colombia,
                "type": "line",
                "color": "black",
                "opacity": 0.03,  # Opacidad ajustada para líneas divisorias más suaves
            }
        ]
    )

    # Ajustes adicionales de diseño
    fig.update_layout(
        title={
            'text': f"Transformadores de Material: {material.capitalize() if material != 'todos' else 'Todos'}",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        margin={"r":0,"t":30,"l":0,"b":0}
    )

    # Mostrar el mapa
    fig.show()


# Para ver todos los transformadores sin importar el material
graficar_transformadores_mapa()


Pruebe que la función funciona corriendola para un ejemplo en el DataFrame `df`

In [ ]:
#EJEMPLO
# Para ver solo los transformadores de cartón
graficar_transformadores_mapa('cartón')

## <font color='8EC044'> **Ejercicio 2** </font>

Diseñe una función que dados el argumento `material` retorne un gráfico de barras que retorne la capacidad de procesamiento de dicho material en los distintos departamentos. La función deberá tener un valor por defecto todos, que muestre todos los transformadores sin impoertar le material que procesan


In [ ]:
import pandas as pd
import plotly.express as px

# Cargar el archivo de transformadores
df = pd.read_csv('datos_transformadores.csv')

def graficar_capacidad_total_por_departamento(material="todos"):
    """
    Función para visualizar la capacidad total de procesamiento en los distintos departamentos,
    con opción de filtrar por un material específico.

    Parámetros:
    - material (str): Tipo de material a mostrar. Puede ser 'plástico', 'cartón', 'vidrio', 'metales',
                      'multimaterial', 'varios', 'papel' o 'todos' (por defecto).
    """
    # Filtrar los datos según el tipo de material
    if material != "todos":
        df_filtrado = df[df['material'] == material]
    else:
        df_filtrado = df

    # Agrupar por departamento y sumar la capacidad de procesamiento total
    capacidad_total_departamento = df_filtrado.groupby('departamento')['capacidad'].sum().reset_index()

    # Crear el gráfico de barras
    fig = px.bar(
        capacidad_total_departamento,
        x='departamento',
        y='capacidad',
        title=f"Capacidad de Procesamiento por Departamento para {material.capitalize() if material != 'todos' else 'Todos los Materiales'}",
        labels={'departamento': 'Departamento', 'capacidad': 'Capacidad de Procesamiento (toneladas)'},
        color_discrete_sequence=["#636EFA"]  # Color del gráfico
    )

    # Mejorar visualización del gráfico
    fig.update_layout(
        xaxis={'categoryorder': 'total descending'},  # Ordenar departamentos por capacidad
        xaxis_tickangle=-45  # Rotación de etiquetas del eje x
    )
    fig.show()

# Ejemplo de uso:
# Para ver la capacidad total de procesamiento de todos los materiales
graficar_capacidad_total_por_departamento()





Pruebe que la función funciona corriendola para un ejemplo en el DataFrame `df`

In [ ]:
# Para ver solo la capacidad de procesamiento de cartón
graficar_capacidad_total_por_departamento('cartón')

